<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-LLM-ZeroToAll/09_4_GraphDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## GraphDB
- https://medium.com/microsoftazure/introducing-graphrag-with-langchain-and-neo4j-90446df17c1e

In [1]:
! pip3 install -qU  markdownify  langchain-upstage==0.1.8rc0 neo4j
! pip3 install langchain-upstage==0.1.8rc0


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY
# ... KEY

In [3]:
# Neo4jVector requires the Neo4j database credentials
import os
url = os.environ["NEO4J_URI"]
username = os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"]

# You can also use environment variables instead of directly passing named parameters
# os.environ["NEO4J_URI"] = "bolt://localhost:7687"
# os.environ["NEO4J_USERNAME"] = "neo4j"
# os.environ["NEO4J_PASSWORD"] = "pleaseletmein"

In [4]:
import warnings

warnings.filterwarnings("ignore")

In [8]:
import pprint as pp
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage
from langchain_experimental.graph_transformers.llm import LLMGraphTransformer

from langchain.globals import set_debug, set_verbose

set_verbose(True)

llm = ChatUpstage(temperature=0)

#initializing the LLMTransformer model
llm_transformer = LLMGraphTransformer(llm=llm)

#transforming documents
from langchain_core.documents import Document

text = """
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for several years;
in fact, Mrs. Dursley pretended she didn't have a sister, because her
sister and her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors would
say if the Potters arrived in the street. The Dursleys knew that the
Potters had a small son, too, but they had never even seen him. This boy
was another good reason for keeping the Potters away; they didn't want
Dudley mixing with a child like that.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)

pp.pprint(graph_documents)

print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you'd expect to be involved in anything strange or mysterious,\nbecause they just didn't hold with such nonsense.\nMr. Dursley was the director of a firm called Grunnings, which made\ndrills. He was a big, beefy man with hardly any neck, although he did\nhave a very large mustache. Mrs. Dursley was thin and blonde and had\nnearly twice the usual amount of neck, which came in very useful as she\nspent so much of her time craning over garden fences, spying on the\nneighbors. The Dursleys had a small son called Dudley and in their\nopinion there was no finer boy anywhere.\nThe Dursleys had everything they wanted, but they also had a secret, and\ntheir greatest fear was that somebody would discover it. They didn't\nthink they could bear it if anyone found 

In [6]:
from langchain_community.graphs.neo4j_graph import Neo4jGraph

graph = Neo4jGraph(url=url, username=username, password=password)

ValueError: Cannot resolve address 3ccd1fbe.databases.neo4j.io:7687

In [ ]:
graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

In [ ]:
from neo4j import GraphDatabase

# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

graph = session.run(cypher).graph()

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

# show the graph resulting from the given Cypher query
# Show in jupiter notebook
# TBA

NameError: name 'session' is not defined

In [ ]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_upstage.embeddings import UpstageEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    url=url, username=username, password=password,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

ValueError: range() arg 3 must not be zero

In [ ]:
query = "Who is Dudley?"

results = vector_index.similarity_search(query, k=1)
print(results[0].page_content)

NameError: name 'vector_index' is not defined

In [ ]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What is Mr. Dursley's job?"})
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:WORKS_FOR]->(j:Job)
WHERE p.name = 'Mr. Dursley'
RETURN j.name
Full Context:
[]

> Finished chain.
{'query': "What is Mr. Dursley's job?", 'result': "I'm sorry, but I don't have the information to answer your question."}


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)

result = qa_chain({"query": "What is Mr. Dursley's job?"})
result["result"]

/Users/hunkim/workspace/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Mr. Dursley is the director of a firm called Grunnings, which makes drills.'